## 术语

要使用 with 语句，首先要明白上下文管理器这一概念。有了上下文管理器，with 语句才能工作。

下面是一组与上下文管理器和with 语句有关的概念。

**上下文管理协议（Context Management Protocol）**：包含方法 `__enter__()` 和 `__exit__()`，支持

该协议的对象要实现这两个方法。

**上下文管理器（Context Manager）**：支持上下文管理协议的对象，这种对象实现了

`__enter__()` 和 `__exit__()` 方法。上下文管理器定义执行 with 语句时要建立的运行时上下文，

负责执行 with 语句块上下文中的进入与退出操作。通常使用 with 语句调用上下文管理器，

也可以通过直接调用其方法来使用。

**运行时上下文（runtime context）**：由上下文管理器创建，通过上下文管理器的 `__enter__()` 和

`__exit__()` 方法实现，`__enter__()` 方法在语句体执行之前进入运行时上下文，`__exit__()` 在

语句体执行完后从运行时上下文退出。with 语句支持运行时上下文这一概念。

**上下文表达式（Context Expression）**：with 语句中跟在关键字 with 之后的表达式，该表达式

要返回一个上下文管理器对象。

语句体（with-body）：with 语句包裹起来的代码块，在执行语句体之前会调用上下文管

理器的 `__enter__()` 方法，执行完语句体之后会执行 `__exit__()` 方法。
 

## 基本语法和工作原理

with 语句的语法格式如下：

清单 1. with 语句的语法格式

```python
with context_expression [as target(s)]:
    with-body
```   
清单 2. 多项表示
```python
with context_expression_1 as A, context_expression_2 as B:
    with-body
```
清单 3. nested函数多项表示
```python
with nested(A(), B(), C()) as (X, Y, Z):
     # with-body code here
```
清单 4.多嵌套
```python
with A() as X:
    with B() as Y:
        with C() as Z:
             # with-body code here
```

1. 执行 context_expression，生成上下文管理器 context_manager

2. 调用上下文管理器的 `__enter__()` 方法；如果使用了 as 子句，则将 `__enter__()` 方法的返回值赋值给 as 子句中的 target(s)

3. 执行语句体 with-body

4. 不管是否执行过程中是否发生了异常，执行上下文管理器的 `__exit__()` 方法，`__exit__()` 方法负责执行“清理”工作，如释放资源等。

5. 如果执行过程中没有出现异常，或者语句体中执行了语句 break/continue/return，则以 None 作为参数调用 `__exit__(None, None, None)` 

6. 如果执行过程中出现异常，则使用 sys.exc_info 得到的异常信息为参数调用 `__exit__(exc_type, exc_value, exc_traceback)`

7. 如果 `__exit__(type, value, traceback)` 返回 False，则会重新抛出异常，让with 之外的语句逻辑来处理异常

open()方法 使用with语句
```python
with open("read.txt","r") as f:
    print(f.readlines())
```

## 自定义with上下文管理器

执行流程如下

1. 自定义类，定义`__enter__()`和`__exit__()`方法

2. 执行with语句块内操作

3. with语句块结束，调用`__exit__()`方法

`__enter__()` ：进入上下文管理器的运行时上下文，在语句体执行前调用。with 语句将该方法的返回值赋值给 as 子句中的 target，如果指定了 as 子句的话

`__exit__(exc_type, exc_value, exc_traceback)` ：退出与上下文管理器相关的运行时上下文，返回一个布尔值表示是否对发生的异常进行处理。参数表示引起退出操作的异常，如果退出时没有发生异常，则3个参数都为None。如果发生异常，返回
True 表示不处理异常，否则会在退出该方法后重新抛出异常以由 with 语句之外的代码逻辑进行处理。如果该方法内部产生异常，则会取代由 statement-body 中语句产生的异常。要处理异常时，不要显示重新抛出异常，即不能重新抛出通过参数传递进来的异常，只需要将返回值设置为 False 就可以了。之后，上下文管理代码会检测是否 `__exit__() `失败来处理异常

In [5]:
class File():
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode

    def __enter__(self):
        self.open_file = open(self.filename, self.mode)
        return self.open_file

    def __exit__(self, *args):
        self.open_file.close()


files = []
for _ in range(10):
    with File('foo.txt', 'w') as infile:
        infile.write('foo')
        files.append(infile)

In [ ]:
from contextlib import contextmanager

@contextmanager
def open_file(path, mode):
    the_file = open(path, mode)
    yield the_file
    the_file.close()

files = []

for x in range(100000):
    with open_file('foo.txt', 'w') as infile:
        files.append(infile)

for f in files:
    if not f.closed:
        print('not closed')

In [ ]:
from contextlib import contextmanager

@contextmanager
def tag(name):
    print("<%s>" % name)
    yield
    print("</%s>" % name)

with tag("h1"):
    print("foo")
"""
<h1>
foo
</h1>
"""

In [1]:
class withm():
    
    def __init__(self,content):
        self.content = content
        print('Init.')

    def __enter__(self):
        print("enter the with block.")
        return self

    def text(self):
        print(self.content)


    def __exit__(self, *args):
        print("exit the with block.")


with withm('orange') as o:
    o.text()

Init.
enter the with block.
orange
exit the with block.


In [2]:
class Sample():
    def __enter__(self):
        print('in enter')
        return self
    def __exit__(self, exc_type, exc_val, exc_tb):
        print ("type: ", exc_type)
        print ("val: ", exc_val)
        print ("tb: ", exc_tb)
    def do_something(self):
        bar = 1 / 0
        return bar + 10
    
with Sample() as sample:
    sample.do_something()

in enter
type:  <class 'ZeroDivisionError'>
val:  division by zero
tb:  <traceback object at 0x000001EB352F0648>


ZeroDivisionError: division by zero

## contextlib 模块

contextmanager用于对**生成器函数**进行装饰。

In [3]:
from contextlib import contextmanager
 
@contextmanager
def demo():
    print ('[Allocate resources]')
    print ('Code before yield-statement executes in __enter__')
    yield ('*** contextmanager demo ***')
    print ('Code after yield-statement executes in __exit__')
    print ('[Free resources]')

with demo() as value:
    print ('Assigned Value: %s' % value)

[Allocate resources]
Code before yield-statement executes in __enter__
Assigned Value: *** contextmanager demo ***
Code after yield-statement executes in __exit__
[Free resources]


1. yield之前的语句在`__enter__()`方法中执行，即在`with context_expression`中执行
2. yield之后的语句在`__exit__()`中执行
3. yield产生的值赋值给as中的value
4. 不同于`__enter__()` / `__exit__()` contextmanager需要在yield语句前后定义，执行**获取**和**清理**操作。

## 上下文管理 closing 实现

上下文管理器会将包装的对象赋值给 as 子句的 target 变量，

同时保证打开的对象在 with-body 执行完后会关闭掉。

closing 上下文管理器包装起来的对象必须提供 close() 方法的定义，

否则执行时会报 AttributeError 错误。

In [4]:
class closing(object):
    # help doc here
    def __init__(self, thing):
        self.thing = thing
    def __enter__(self):
        return self.thing
    def __exit__(self, *exc_info):
        self.thing.close()
        
class ClosingDemo(object):
    def __init__(self):
        self.acquire()
    def acquire(self):
        print ('Acquire resources.')
    def free(self):
        print ('Clean up any resources acquired.')
    def close(self):
        self.free()
        
with closing(ClosingDemo()):
    print ('Using resources')

Acquire resources.
Using resources
Clean up any resources acquired.


closing 适用于提供了 close() 实现的对象，比如网络连接、数据库连接等，也可以在自定义类时通过接口 close() 来执行所需要的资源“清理”工作

## ContextDecorator类继承

In [2]:
from contextlib import ContextDecorator

class makeparagraph(ContextDecorator):
    def __enter__(self):
        print('<p>')
        return self

    def __exit__(self, *exc):
        print('</p>')
        return False

@makeparagraph()
def emit_html():
    print('Here is some non-HTML')

emit_html()

<p>
Here is some non-HTML
</p>
